In [2]:
%load_ext autoreload
%autoreload 2

import parse_data.prepare_data as prepare_data
import parse_data.flip_rotate_trajectories as rotate
from analysis.response_times import calculate_response_times, plot_response_times
import data_extraction.get_indices as get_indices
import plotting.plot_octagon as plot_octagon
import globals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import analysis.loser_inferred_choice as loser_inferred_choice
import seaborn as sns
import data_strings
import data_extraction.trial_list_filters as trial_list_filters


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Umbrella notebook to collect all analysis on individuals

#### List of analyses:
- Solo performance (score per unit time)
- Social performance (score per unit time)
- P(win) social
- Sensory effects (P(H|H first vis) - P(H|L first vis)) (solo and social)
- Sensory effects difference (comparing solo and social)
- Performance across wall sep 
- Sensory effects across wall sep
- Game control ability


#### Dictionary structure A: Individual participant analysis is the lowest level

In [ ]:
analysis_results = {
    session_id: {
        player_id: {

            # solo analysis
            'solo': {
                'placeholder_data1': None,
            },

            # social analysis
            'social': {
                'placeholder_data2': None,
            },

            # solo-social comparison analysis
            'difference': {
                'placeholder_data3': None,

            }
        }
        for player_id in placeholder[session_id]
    }
    for session_id in placeholder2
}

#### Dictionary structure B: Analysis remains in array format 

In [ ]:
analysis_results = {
    "social": {
        "performance": None
        "sensory_effects": None
    },

    "solo": {
        "performance": None
        "sensory_effects": None
    },

    "difference": {
        "placeholder": None
    }

}

### Alternative approach: Use object-oriented structure because of the complexity

In [ ]:
# low-level class for an individual analysis, with name and dictionary of results
class Analysis:
    def __init__(self, name, results):
        self.name = name
        self.results = results

    def get_result(self, key):
        return self.results.get(key, None)
    
    def add_result(self, key, value):
        self.results[key] = value

# instance made for each participant in an experiment
class Participant:
    def __init__(self, participant_id):
        self.participant_id = participant_id
        self.analyses = {}

    def add_analysis(self, analysis_name, results):
        # Create an Analysis object and add it to the dict
        self.analyses[analysis_name] = Analysis(analysis_name, results)

    def get_analysis(self, analysis_name):
        return self.analyses.get(analysis_name, None)
    
    def add_analysis_object(self, analysis_obj):
        # use to add a pre-existing Analysis object
        self.analyses[analysis_obj.name] = analysis_obj

# instance made for each experiment
class Experiment:
    def __init__(self, experiment_id):
        self.experiment_id = experiment_id
        self.participants = {}

    def add_participant(self, participant_id):
        if participant_id not in self.participants:
            self.participants[participant_id] = Participant(participant_id)
    
    def get_participant(self, participant_id):
        if participant_id in self.participants:
            return self.participants[participant_id]
        else:
            return None